In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from sklearn.metrics import classification_report
from pprint import pprint

class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MyLSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # LSTM layer
        out, _ = self.lstm(x)

        # Take the output from the last time step
        out = out[:, -1, :]

        # Fully connected layer
        out = self.fc(out)

        return out

# 모델 생성
input_size = 70  # 입력 데이터의 피쳐 수
hidden_size = 32  # LSTM의 hidden state 크기
num_layers = 1    # LSTM 레이어의 수
output_size = 1   # 출력의 크기

model = MyLSTM(input_size, hidden_size, num_layers, output_size)

print(model)

MyLSTM(
  (lstm): LSTM(70, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


In [2]:
testX = np.load('testX.npy')
testY = np.load('testY.npy')
trainX = np.load('trainX.npy')
trainY = np.load('trainY.npy')

trainX_tensor = torch.FloatTensor(trainX)
trainY_tensor = torch.FloatTensor(trainY)
testX_tensor = torch.FloatTensor(testX)
testY_tensor = torch.FloatTensor(testY)

In [3]:

# 손실 함수 및 최적화 알고리즘 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# 훈련
num_epochs = 10

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(trainX_tensor.float())
    # 손실 계산
    loss = criterion(outputs, trainY_tensor.float())
    # Backward pass 및 최적화
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [4]:

predY = model(testX_tensor.float()).data.numpy()
print(predY.shape)
print(predY.max())
print(predY.min())

(1895, 1)
0.16119134
-0.8117335


In [5]:
for i in range(len(predY)):
    if predY[i] > 0:
        predY[i] = 1
    else:
        predY[i] = 0
result= classification_report(testY_tensor, predY, output_dict=True) 
pprint(result)

{'0.0': {'f1-score': 0.6776268763402429,
         'precision': 0.5169029443838604,
         'recall': 0.983402489626556,
         'support': 964.0},
 '1.0': {'f1-score': 0.0907258064516129,
         'precision': 0.7377049180327869,
         'recall': 0.04833512352309345,
         'support': 931.0},
 'accuracy': 0.5240105540897098,
 'macro avg': {'f1-score': 0.3841763413959279,
               'precision': 0.6273039312083237,
               'recall': 0.5158688065748247,
               'support': 1895.0},
 'weighted avg': {'f1-score': 0.38928656179337506,
                  'precision': 0.6253813810419874,
                  'recall': 0.5240105540897098,
                  'support': 1895.0}}
